In [26]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score


In [27]:
train_data = pd.read_csv("../data/train.csv")
train_data.head()


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [28]:
test_data = pd.read_csv("../data/test.csv")
test_data.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [ ]:


# Perform standard preprocessing (as in previous code)
def prepare_data(train_data, test_data):
    # [Previous preprocessing steps remain the same]
    train_data['Title'] = train_data['Name'].str.extract(' ([A-Za-z]+)\.', expand=False)
    test_data['Title'] = test_data['Name'].str.extract(' ([A-Za-z]+)\.', expand=False)
    
    rare_titles = ['Lady', 'Countess', 'Capt', 'Col', 'Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona']
    train_data['Title'] = train_data['Title'].replace(rare_titles, 'Rare')
    test_data['Title'] = test_data['Title'].replace(rare_titles, 'Rare')
    
    train_data['Title'] = train_data['Title'].replace(['Mlle', 'Ms'], 'Miss')
    train_data['Title'] = train_data['Title'].replace('Mme', 'Mrs')
    test_data['Title'] = test_data['Title'].replace(['Mlle', 'Ms'], 'Miss')
    test_data['Title'] = test_data['Title'].replace('Mme', 'Mrs')
    title_age_median = train_data.groupby('Title')['Age'].median()
    train_data['Age'] = train_data.apply(
        lambda row: title_age_median[row['Title']] if pd.isna(row['Age']) else row['Age'], 
        axis=1
    )
    test_data['Age'] = test_data.apply(
        lambda row: title_age_median.get(row['Title'], train_data['Age'].median()) 
        if pd.isna(row['Age']) else row['Age'], 
        axis=1
    )
    
    train_data['Embarked'] = train_data['Embarked'].fillna(train_data['Embarked'].mode()[0])
    train_data['Fare'] = train_data['Fare'].fillna(train_data['Fare'].median())
    test_data['Fare'] = test_data['Fare'].fillna(test_data['Fare'].median())
    
    return train_data, test_data

train_data, test_data = prepare_data(train_data, test_data)
train_data['FamilySize'] = train_data['SibSp'] + train_data['Parch'] + 1
train_data['IsAlone'] = (train_data['FamilySize'] == 1).astype(int)
test_data['FamilySize'] = test_data['SibSp'] + test_data['Parch'] + 1
test_data['IsAlone'] = (test_data['FamilySize'] == 1).astype(int)



X = pd.get_dummies(train_data[["Pclass", "Sex",  "SibSp", "Parch", "Embarked", "Title", "FamilySize","IsAlone","Fare"]])
y = train_data["Survived"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

rf_model = RandomForestClassifier(n_estimators=100, max_depth=3, random_state=43)
rf_model.fit(X_train, y_train)




In [30]:
print(rf_model.score(X_test,y_test))

0.8435754189944135
